In [1]:
import pandas as pd

df = pd.read_csv(r'single_label.csv') #Change name here
print(df.columns)


Index(['text', 'label'], dtype='object')


In [2]:
print(df['label'].value_counts())


label
T1027        683
T1140        455
T1059.003    345
T1055        283
T1105        262
T1106        201
T1078        170
T1071.001    146
T1090        139
T1082        137
T1003.001    107
T1053.005    106
T1112        104
T1083         98
T1562.001     95
T1021.001     94
T1204.002     87
T1070.004     86
T1566.001     86
T1041         82
T1057         82
T1574.002     81
T1047         75
T1036.005     68
T1056.001     65
T1110         64
T1005         64
T1547.001     63
T1570         57
T1016         55
T1218.011     53
T1219         52
T1573.001     52
T1190         51
T1095         50
T1543.003     50
T1033         49
T1113         48
T1518.001     36
T1548.002     27
T1012         26
T1074.001     25
T1569.002     23
T1484.001     23
T1552.001     20
T1564.001     17
T1210         16
T1068         10
T1072         10
T1557.001      4
Name: count, dtype: int64


In [ ]:
import torch
import pandas as pd
import numpy as np
import torch
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader

from transformers import AutoTokenizer, AutoModelForMaskedLM, BertConfig, AutoModel,RobertaTokenizer,RobertaModel,RobertaConfig
from sklearn.preprocessing import LabelEncoder
import time
from time import sleep

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

pretrained_model = AutoModelForMaskedLM.from_pretrained("bert-base-uncased")

config = BertConfig.from_pretrained("bert-base-uncased", output_hidden_states=True)

"""Set the Runtime to GPU and check and set cuda availability with the following snippet"""

# Setting up the device for GPU usage

from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'
device

LABELS = len(df['label'].unique())
print("LABELS ================== :", LABELS)

"""Substitute dataset file name with your own"""
def experiment(seed):
	start_time = time.time()  # Record the start time
	df = pd.read_csv(r'single_label.csv') #Change name here

	LABELS = len(df['label'].unique())

	#Encoding labels
	encoder = LabelEncoder()
	encoder.fit(df['label'])
	df['enc_Domain'] = encoder.transform(df['label'])

	df = df[['label', 'text', 'enc_Domain']]
	LABELS

	"""# Model"""

	# Defining some key variables that will be used later on in the training
	MAX_LEN = 512
	TRAIN_BATCH_SIZE = 16
	VALID_BATCH_SIZE = 16
	EPOCHS = 20
	LEARNING_RATE = 1e-05

	class Triage(Dataset):
		def __init__(self, dataframe, tokenizer, max_len):
			self.len = len(dataframe)
			self.data = dataframe
			self.tokenizer = tokenizer
			self.max_len = max_len

		def __getitem__(self, index):
			sentence = str(self.data.text[index])
			sentence = " ".join(sentence.split())
			inputs = self.tokenizer.encode_plus(
				sentence,
				None,
				add_special_tokens=True,
				max_length=self.max_len,
				padding='max_length',
				return_token_type_ids=True,
				truncation=True
			)
			ids = inputs['input_ids']
			mask = inputs['attention_mask']

			if 'enc_Domain' not in self.data:
				return {
				'ids': torch.tensor(ids, dtype=torch.long),
				'mask': torch.tensor(mask, dtype=torch.long)
				}

			return {
				'ids': torch.tensor(ids, dtype=torch.long),
				'mask': torch.tensor(mask, dtype=torch.long),
				'targets': torch.tensor(self.data.enc_Domain[index], dtype=torch.long)
			}

		def __len__(self):
			return self.len
	experiment_seed = seed
	#Split dataset into train and validation
	train_indices, test_indices = train_test_split(list(range(len(df.enc_Domain))), random_state = seed, test_size=0.2, stratify=df.enc_Domain)

	train_dataset = df.copy().drop(test_indices).reset_index(drop=True)
	test_dataset = df.copy().drop(train_indices).reset_index(drop=True)


	print("FULL Dataset: {}".format(df.shape))
	print("TRAIN Dataset: {}".format(train_dataset.shape))
	print("TEST Dataset: {}".format(test_dataset.shape))

	training_set = Triage(train_dataset, tokenizer, MAX_LEN)
	testing_set = Triage(test_dataset, tokenizer, MAX_LEN)

	train_params = {'batch_size': TRAIN_BATCH_SIZE,
		        'shuffle': True,
		        'num_workers': 0
		        }

	test_params = {'batch_size': VALID_BATCH_SIZE,
		        'shuffle': True,
		        'num_workers': 0
		        }

	training_loader = DataLoader(training_set, **train_params)
	testing_loader = DataLoader(testing_set, **test_params)

	# Creating the customized model, by adding a drop out and a dense layer on top of distil bert to get the final output for the model.

	class BERTClass(torch.nn.Module):
		def __init__(self, pretrained_model_name: str, num_classes: int = None, dropout: float = 0.5):
			super().__init__()
			config = BertConfig.from_pretrained(pretrained_model_name, output_hidden_states=True)
			self.model = AutoModelForMaskedLM.from_pretrained(pretrained_model_name, config=config).base_model #pick only the main body of the model
			#for param in self.model.parameters():
			#param.requires_grad = False
			self.pre_classifier = torch.nn.Linear(768, 768)
			self.dropout = torch.nn.Dropout(dropout)
			self.classifier = torch.nn.Linear(768, num_classes)

		def forward(self, input_ids, attention_mask):
			output_1 = self.model(input_ids=input_ids, attention_mask=attention_mask, output_hidden_states=True)
			hidden_state = output_1[0]
			pooler = hidden_state[:, 0]
			pooler = self.pre_classifier(pooler)
			pooler = torch.nn.ReLU()(pooler)
			pooler = self.dropout(pooler)
			output = self.classifier(pooler)
			return output

	#LOAD
	model = BERTClass("bert-base-uncased", LABELS)


	device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
	model.to(device)

	# Creating the loss function and optimizer
	loss_function = torch.nn.CrossEntropyLoss()
	optimizer = torch.optim.Adam(params =  model.parameters(), lr=LEARNING_RATE)

	# Function to calcuate the accuracy of the model
	def calcuate_accu(big_idx, targets):
		n_correct = (big_idx==targets).sum().item()
		return n_correct

	torch.cuda.empty_cache()

	# Defining the training function on the 80% of the dataset for tuning the secbert model
	def train(epoch):
		tr_loss = 0
		n_correct = 0
		nb_tr_steps = 0
		examples = len(train_dataset)
		losses = [None] * len(training_loader)
		model.train()
		#loop = tqdm(enumerate(training_loader), total=len(training_loader), leave=False)
		for i, data in enumerate(training_loader, 0):
			#print(i)
			ids = data['ids'].to(device, dtype = torch.long)
			mask = data['mask'].to(device, dtype = torch.long)
			targets = data['targets'].to(device, dtype = torch.long)

			outputs = model(ids, mask)
			loss = loss_function(outputs, targets)

			losses[i] = loss.item()
			optimizer.zero_grad()
			loss.backward()
			# # When using GPU
			optimizer.step()

		print(f"Cost at epoch {epoch} is {sum(losses)/len(losses):.5f}")
		return


	for epoch in range(EPOCHS):
		train(epoch)

	def check_accuracy(loader, model):

		#pred = []
		num_correct = 0
		num_samples = 0
		model.eval()

		with torch.no_grad():
			for i, data in enumerate(loader, 0):
				x = data['ids'].to(device, dtype = torch.long)
				mask = data['mask'].to(device, dtype = torch.long)
				y = data['targets'].to(device, dtype = torch.long)

				scores = model(x, mask)
				_, predictions = scores.max(1)
				num_correct += (predictions == y).sum()
				num_samples += predictions.size(0)
				pred.append(predictions.cpu().numpy())
				y_test.append(y.cpu().numpy())


			print(f"Got {num_correct} / {num_samples} with accuracy {float(num_correct)/float(num_samples)*100:.2f}")

	pred=[]
	y_pred =[]
	y_test =[]
	y =[]

	"""Save the model for further tests"""

	check_accuracy(testing_loader, model)

	for i in range(len(pred)):
		t = pred[i]
		for j in range(len(t)):
			y_pred.append(t[j])

	for i in range(len(y_test)):
		t = y_test[i]
		for j in range(len(t)):
			y.append(t[j])

	from sklearn.metrics import classification_report,accuracy_score
	print(classification_report(y, y_pred))

	labels = list(encoder.inverse_transform([0,1,2,3,4,5]))

	from sklearn.metrics import classification_report

	# Assuming you have true labels (y_true) and predicted labels (y_pred)
	# Calculate classification report
	report = classification_report(y, y_pred, target_names = labels, digits=4, output_dict=True)

	# Access the weighted F1 score, recall, and precision
	f1_weighted = report['weighted avg']['f1-score']
	recall_weighted = report['weighted avg']['recall']
	precision_weighted = report['weighted avg']['precision']
	accuracy=accuracy_score(y,y_pred)
	# Access the macro-averaged F1 score, recall, and precision
	f1_macro = report['macro avg']['f1-score']
	recall_macro = report['macro avg']['recall']
	precision_macro = report['macro avg']['precision']

	# Print the results
	print("Weighted Precision:", precision_weighted)
	print("Weighted Recall:", recall_weighted)
	print("Weighted F1 Score:", f1_weighted)

	# Print the results
	print("Macro Precision:", f1_macro)
	print("Macro Recall:", recall_macro)
	print("Macro F1 Score:", precision_macro)
	print("Accuracy: ", accuracy)

	from sklearn.metrics import confusion_matrix
	import matplotlib.pyplot as plt
	import seaborn as sns
	fig, ax = plt.subplots(figsize=(7, 7))
	cm_array = confusion_matrix(y, y_pred)
	cm_labels = np.unique(labels)
	cm_array_df = pd.DataFrame(cm_array, index=cm_labels)
	sns.heatmap(cm_array_df, annot=True,
		    cbar=False, fmt='1d', cmap='Blues', ax=ax)
	ax.set_title('Confusion Matrix', loc='left', fontsize=16)
	ax.set_xlabel('Predicted',fontsize=16)
	ax.set_ylabel('Actual',fontsize=16)
	image_name = f"confusion_matrix_{experiment_seed}.png"
	plt.savefig(image_name)
	# Calculate and print execution time
	end_time = time.time()
	execution_time = end_time - start_time
	print(f"Execution Time: {execution_time} seconds")
	# Appending classification report, confusion matrix text, and confusion matrix image path to the same text file
	with open("BERTBase_result.txt", "a") as file:
		file.write("\n\nBERTBase Result:\n\n")
		file.write(f"\n\nSeed Value: {experiment_seed}")
		file.write("\n\nClassification Report:\n\n")
		file.write(classification_report(y, y_pred, target_names = labels))
		file.write(f"\n\nWeighted Precision: {precision_weighted}")
		file.write(f"\n\nWeighted Recall: {recall_weighted}")
		file.write(f"\n\nWeighted F1 Score: {f1_weighted}")
		file.write(f"\n\Macro Precision: {precision_macro}")
		file.write(f"\n\Macro Recall: {recall_macro}")
		file.write(f"\n\Macro F1 Score: {f1_macro}")
		file.write(f"\n\nAccuracy: {accuracy}")
		file.write("\n\nConfusion Matrix Image Path:\n\n")
		file.write(image_name)
		file.write(f"\n\nExecution Time: {execution_time} seconds")
experiment(2092)
experiment(22)
experiment(1517)
experiment(919)
experiment(2455)
experiment(65)
experiment(2167)
experiment(246)
experiment(1159)
experiment(2679)


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


LABELS ================== : 51
FULL Dataset: (5086, 3)
TRAIN Dataset: (4068, 3)
TEST Dataset: (1018, 3)


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Cost at epoch 0 is 3.50133
Cost at epoch 1 is 2.60356
Cost at epoch 2 is 1.87790
Cost at epoch 3 is 1.36521
Cost at epoch 4 is 1.01949


In [ ]:
import torch
import pandas as pd
import numpy as np
import torch
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader

from transformers import AutoTokenizer, AutoModelForMaskedLM, BertConfig, AutoModel,RobertaTokenizer,RobertaModel,RobertaConfig
from sklearn.preprocessing import LabelEncoder
import time
from time import sleep

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

pretrained_model = AutoModelForMaskedLM.from_pretrained("bert-base-uncased")

config = BertConfig.from_pretrained("bert-base-uncased", output_hidden_states=True)

"""Set the Runtime to GPU and check and set cuda availability with the following snippet"""

# Setting up the device for GPU usage

from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'
device

df = pd.read_csv(r'single_label.csv') #Change name here
LABELS = len(df['label'].unique())
print("========= UNIQUE LABELS =========== : ", LABELS)

"""Substitute dataset file name with your own"""
def experiment(seed):
	start_time = time.time()  # Record the start time
	df = pd.read_csv(r'single_label.csv') #Change name here

	LABELS = len(df['label'].unique())

	#Encoding labels
	encoder = LabelEncoder()
	encoder.fit(df['label'])
	df['enc_Domain'] = encoder.transform(df['label'])

	df = df[['label', 'text', 'enc_Domain']]
	LABELS

	"""# Model"""

	# Defining some key variables that will be used later on in the training
	MAX_LEN = 512
	TRAIN_BATCH_SIZE = 16
	VALID_BATCH_SIZE = 16
	EPOCHS = 20
	LEARNING_RATE = 1e-05

	class Triage(Dataset):
		def __init__(self, dataframe, tokenizer, max_len):
			self.len = len(dataframe)
			self.data = dataframe
			self.tokenizer = tokenizer
			self.max_len = max_len

		def __getitem__(self, index):
			sentence = str(self.data.text[index])
			sentence = " ".join(sentence.split())
			inputs = self.tokenizer.encode_plus(
				sentence,
				None,
				add_special_tokens=True,
				max_length=self.max_len,
				padding='max_length',
				return_token_type_ids=True,
				truncation=True
			)
			ids = inputs['input_ids']
			mask = inputs['attention_mask']

			if 'enc_Domain' not in self.data:
				return {
				'ids': torch.tensor(ids, dtype=torch.long),
				'mask': torch.tensor(mask, dtype=torch.long)
				}

			return {
				'ids': torch.tensor(ids, dtype=torch.long),
				'mask': torch.tensor(mask, dtype=torch.long),
				'targets': torch.tensor(self.data.enc_Domain[index], dtype=torch.long)
			}

		def __len__(self):
			return self.len
	experiment_seed = seed
	#Split dataset into train and validation
	train_indices, test_indices = train_test_split(list(range(len(df.enc_Domain))), random_state = seed, test_size=0.2, stratify=df.enc_Domain)

	train_dataset = df.copy().drop(test_indices).reset_index(drop=True)
	test_dataset = df.copy().drop(train_indices).reset_index(drop=True)


	print("FULL Dataset: {}".format(df.shape))
	print("TRAIN Dataset: {}".format(train_dataset.shape))
	print("TEST Dataset: {}".format(test_dataset.shape))

	training_set = Triage(train_dataset, tokenizer, MAX_LEN)
	testing_set = Triage(test_dataset, tokenizer, MAX_LEN)

	train_params = {'batch_size': TRAIN_BATCH_SIZE,
		        'shuffle': True,
		        'num_workers': 0
		        }

	test_params = {'batch_size': VALID_BATCH_SIZE,
		        'shuffle': True,
		        'num_workers': 0
		        }

	training_loader = DataLoader(training_set, **train_params)
	testing_loader = DataLoader(testing_set, **test_params)

	# Creating the customized model, by adding a drop out and a dense layer on top of distil bert to get the final output for the model.

	class BERTClass(torch.nn.Module):
		def __init__(self, pretrained_model_name: str, num_classes: int = None, dropout: float = 0.5):
			super().__init__()
			config = BertConfig.from_pretrained(pretrained_model_name, output_hidden_states=True)
			self.model = AutoModelForMaskedLM.from_pretrained(pretrained_model_name, config=config).base_model #pick only the main body of the model
			#for param in self.model.parameters():
			#param.requires_grad = False
			self.pre_classifier = torch.nn.Linear(768, 768)
			self.dropout = torch.nn.Dropout(dropout)
			self.classifier = torch.nn.Linear(768, num_classes)

		def forward(self, input_ids, attention_mask):
			output_1 = self.model(input_ids=input_ids, attention_mask=attention_mask, output_hidden_states=True)
			hidden_state = output_1[0]
			pooler = hidden_state[:, 0]
			pooler = self.pre_classifier(pooler)
			pooler = torch.nn.ReLU()(pooler)
			pooler = self.dropout(pooler)
			output = self.classifier(pooler)
			return output

	#LOAD
	model = BERTClass("bert-base-uncased", LABELS)


	device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
	model.to(device)

	# Creating the loss function and optimizer
	loss_function = torch.nn.CrossEntropyLoss()
	optimizer = torch.optim.Adam(params =  model.parameters(), lr=LEARNING_RATE)

	# Function to calcuate the accuracy of the model
	def calcuate_accu(big_idx, targets):
		n_correct = (big_idx==targets).sum().item()
		return n_correct

	torch.cuda.empty_cache()

	# Defining the training function on the 80% of the dataset for tuning the secbert model
	def train(epoch):
		tr_loss = 0
		n_correct = 0
		nb_tr_steps = 0
		examples = len(train_dataset)
		losses = [None] * len(training_loader)
		model.train()
		#loop = tqdm(enumerate(training_loader), total=len(training_loader), leave=False)
		for i, data in enumerate(training_loader, 0):
			#print(i)
			ids = data['ids'].to(device, dtype = torch.long)
			mask = data['mask'].to(device, dtype = torch.long)
			targets = data['targets'].to(device, dtype = torch.long)

			outputs = model(ids, mask)
			loss = loss_function(outputs, targets)

			losses[i] = loss.item()
			optimizer.zero_grad()
			loss.backward()
			# # When using GPU
			optimizer.step()

		print(f"Cost at epoch {epoch} is {sum(losses)/len(losses):.5f}")
		return


	for epoch in range(EPOCHS):
		train(epoch)

	def check_accuracy(loader, model):

		#pred = []
		num_correct = 0
		num_samples = 0
		model.eval()

		with torch.no_grad():
			for i, data in enumerate(loader, 0):
				x = data['ids'].to(device, dtype = torch.long)
				mask = data['mask'].to(device, dtype = torch.long)
				y = data['targets'].to(device, dtype = torch.long)

				scores = model(x, mask)
				_, predictions = scores.max(1)
				num_correct += (predictions == y).sum()
				num_samples += predictions.size(0)
				pred.append(predictions.cpu().numpy())
				y_test.append(y.cpu().numpy())


			print(f"Got {num_correct} / {num_samples} with accuracy {float(num_correct)/float(num_samples)*100:.2f}")

	pred=[]
	y_pred =[]
	y_test =[]
	y =[]

	"""Save the model for further tests"""

	check_accuracy(testing_loader, model)

	for i in range(len(pred)):
		t = pred[i]
		for j in range(len(t)):
			y_pred.append(t[j])

	for i in range(len(y_test)):
		t = y_test[i]
		for j in range(len(t)):
			y.append(t[j])

	from sklearn.metrics import classification_report,accuracy_score
	print(classification_report(y, y_pred))

	labels = list(encoder.inverse_transform([0,1,2,3,4,5]))

	from sklearn.metrics import classification_report

	# Assuming you have true labels (y_true) and predicted labels (y_pred)
	# Calculate classification report
	report = classification_report(y, y_pred, target_names = labels, digits=4, output_dict=True)

	# Access the weighted F1 score, recall, and precision
	f1_weighted = report['weighted avg']['f1-score']
	recall_weighted = report['weighted avg']['recall']
	precision_weighted = report['weighted avg']['precision']
	accuracy=accuracy_score(y,y_pred)
	# Access the macro-averaged F1 score, recall, and precision
	f1_macro = report['macro avg']['f1-score']
	recall_macro = report['macro avg']['recall']
	precision_macro = report['macro avg']['precision']

	# Print the results
	print("Weighted Precision:", precision_weighted)
	print("Weighted Recall:", recall_weighted)
	print("Weighted F1 Score:", f1_weighted)

	# Print the results
	print("Macro Precision:", f1_macro)
	print("Macro Recall:", recall_macro)
	print("Macro F1 Score:", precision_macro)
	print("Accuracy: ", accuracy)

	from sklearn.metrics import confusion_matrix
	import matplotlib.pyplot as plt
	import seaborn as sns
	fig, ax = plt.subplots(figsize=(7, 7))
	cm_array = confusion_matrix(y, y_pred)
	cm_labels = np.unique(labels)
	cm_array_df = pd.DataFrame(cm_array, index=cm_labels)
	sns.heatmap(cm_array_df, annot=True,
		    cbar=False, fmt='1d', cmap='Blues', ax=ax)
	ax.set_title('Confusion Matrix', loc='left', fontsize=16)
	ax.set_xlabel('Predicted',fontsize=16)
	ax.set_ylabel('Actual',fontsize=16)
	image_name = f"confusion_matrix_{experiment_seed}.png"
	plt.savefig(image_name)
	# Calculate and print execution time
	end_time = time.time()
	execution_time = end_time - start_time
	print(f"Execution Time: {execution_time} seconds")
	# Appending classification report, confusion matrix text, and confusion matrix image path to the same text file
	with open("BERTBase_result.txt", "a") as file:
		file.write("\n\nBERTBase Result:\n\n")
		file.write(f"\n\nSeed Value: {experiment_seed}")
		file.write("\n\nClassification Report:\n\n")
		file.write(classification_report(y, y_pred, target_names = labels))
		file.write(f"\n\nWeighted Precision: {precision_weighted}")
		file.write(f"\n\nWeighted Recall: {recall_weighted}")
		file.write(f"\n\nWeighted F1 Score: {f1_weighted}")
		file.write(f"\n\Macro Precision: {precision_macro}")
		file.write(f"\n\Macro Recall: {recall_macro}")
		file.write(f"\n\Macro F1 Score: {f1_macro}")
		file.write(f"\n\nAccuracy: {accuracy}")
		file.write("\n\nConfusion Matrix Image Path:\n\n")
		file.write(image_name)
		file.write(f"\n\nExecution Time: {execution_time} seconds")
experiment(2092)
experiment(22)
experiment(1517)
experiment(919)
experiment(2455)
experiment(65)
experiment(2167)
experiment(246)
experiment(1159)
experiment(2679)


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


========= UNIQUE LABELS =========== :  51
FULL Dataset: (5086, 3)
TRAIN Dataset: (4068, 3)
TEST Dataset: (1018, 3)


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
